In [1]:
import pyspark
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local") \
    .config('spark.sql.autoBroadcastJoinThreshold', 0) \
    .config('spark.sql.adaptive.enabled', 'false') \
    .getOrCreate()
spark

#### Spark UI:
- http://127.0.0.1:4040/stages/

In [99]:
!du datasets/*

12	datasets/US_category_id.json
71108	datasets/UScomments.csv
2916	datasets/USvideos.csv


In [19]:
videos = spark.read.option('header', 'true').option("inferSchema", "true").csv('datasets/USvideos.csv')
# videos.show(3, False, True)
videos.limit(3).toPandas().T

,0,1,2
video_id,XpVt6Z1Gjjo,K4wEI5zhHB0,cLdxuaxaQwc
title,1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED Y...,iPhone X — Introducing iPhone X — Apple,My Response
channel_title,Logan Paul Vlogs,Apple,PewDiePie
category_id,24,28,22
tags,logan paul vlog|logan paul|logan|paul|olympics...,Apple|iPhone 10|iPhone Ten|iPhone|Portrait Lig...,[none]
views,4394029,7860119,5845909
likes,320053,185853,576597
dislikes,5931,26679,39774
comment_total,46245,0,170708
thumbnail_link,https://i.ytimg.com/vi/XpVt6Z1Gjjo/default.jpg,https://i.ytimg.com/vi/K4wEI5zhHB0/default.jpg,https://i.ytimg.com/vi/cLdxuaxaQwc/default.jpg


In [46]:
comments_schema = StructType([ \
    StructField("video_id", StringType(), True), \
    StructField("comment_text", StringType(), True), \
    StructField("likes", IntegerType(), True), \
    StructField("replies", IntegerType(), True)])
comments = spark.read.option('header', 'true').option("mode", "DROPMALFORMED").schema(comments_schema).csv('datasets/UScomments.csv')
(comments
 # .where("likes < 0")
 .limit(3).toPandas()
)

,video_id,comment_text,likes,replies
0,XpVt6Z1Gjjo,Logan Paul it's yo big day ‼️‼️‼️,4,0
1,XpVt6Z1Gjjo,I've been following you from the start of your...,3,0
2,XpVt6Z1Gjjo,Say hi to Kong and maverick for me,3,0


1 scored_videos - датасет на USvideos.csv
 - с добавлением колонки, содержащей скор (показатель качества) видео: ФОРМУЛА должна включать в себя
     - просмотры,
     - лайки,дизлайки видео,
     - лайки и дизлайки к комментариям к видео
       
2 categories_score - по категориям, в котором поля: 
 - Название категории (не id) -  в US_category_id.json
 - Медиана показателя score из scored_videos по каждой категории

3 popular_tags - по самым популярным тэгам
  - название тэга + количество видео с этим тэгом
    ! тэги лежат строкой в поле tags
      - Scala-функцию для разбиения тегов: 
        - Но напишите свою UDF-функцию разбиения строки на тэги
        - и сравните время работы с её Scala-версией.
          - Можно замерять своими силами,
            - а можно воспользоваться библиотекой timeit
            - функции Spark из пакета pyspark.sq.functions использовать нельзя,
              - нужно написать свою функцию.

In [34]:
videos.limit(3).toPandas().T

,0,1,2
video_id,XpVt6Z1Gjjo,K4wEI5zhHB0,cLdxuaxaQwc
title,1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED Y...,iPhone X — Introducing iPhone X — Apple,My Response
channel_title,Logan Paul Vlogs,Apple,PewDiePie
category_id,24,28,22
tags,logan paul vlog|logan paul|logan|paul|olympics...,Apple|iPhone 10|iPhone Ten|iPhone|Portrait Lig...,[none]
views,4394029,7860119,5845909
likes,320053,185853,576597
dislikes,5931,26679,39774
comment_total,46245,0,170708
thumbnail_link,https://i.ytimg.com/vi/XpVt6Z1Gjjo/default.jpg,https://i.ytimg.com/vi/K4wEI5zhHB0/default.jpg,https://i.ytimg.com/vi/cLdxuaxaQwc/default.jpg


In [100]:
videos.printSchema()
comments.printSchema()
comments.limit(3).toPandas()

root
 |-- video_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- channel_title: string (nullable = true)
 |-- category_id: integer (nullable = true)
 |-- tags: string (nullable = true)
 |-- views: integer (nullable = true)
 |-- likes: integer (nullable = true)
 |-- dislikes: integer (nullable = true)
 |-- comment_total: integer (nullable = true)
 |-- thumbnail_link: string (nullable = true)
 |-- date: string (nullable = true)

root
 |-- video_id: string (nullable = true)
 |-- comment_text: string (nullable = true)
 |-- likes: integer (nullable = true)
 |-- replies: integer (nullable = true)



,video_id,comment_text,likes,replies
0,XpVt6Z1Gjjo,Logan Paul it's yo big day ‼️‼️‼️,4,0
1,XpVt6Z1Gjjo,I've been following you from the start of your...,3,0
2,XpVt6Z1Gjjo,Say hi to Kong and maverick for me,3,0


In [98]:
coalesce??

Signature: coalesce(*cols: 'ColumnOrName') -> pyspark.sql.column.Column
Source:   
@try_remote_functions
def coalesce(*cols: "ColumnOrName") -> Column:
    """Returns the first column that is not null.

    .. versionadded:: 1.4.0

    .. versionchanged:: 3.4.0
        Supports Spark Connect.

    Parameters
    ----------
    cols : :class:`~pyspark.sql.Column` or str
        list of columns to work on.

    Returns
    -------
    :class:`~pyspark.sql.Column`
        value of the first column that is not null.

    Examples
    --------
    >>> cDf = spark.createDataFrame([(None, None), (1, None), (None, 2)], ("a", "b"))
    >>> cDf.show()
    +----+----+
    |   a|   b|
    +----+----+
    |null|null|
    |   1|null|
    |null|   2|
    +----+----+

    >>> cDf.select(coalesce(cDf["a"], cDf["b"])).show()
    +--------------+
    |coalesce(a, b)|
    +--------------+
    |          null|
    |             1|
    |             2|
    +--------------+

    >>> cDf.select('*', coalesce(

In [115]:
# 1 scored_videos - датасет на USvideos.csv
#  - с добавлением колонки, содержащей скор (показатель качества) видео:
#    - ФОРМУЛА должна включать в себя
#      - просмотры,
#      - лайки, дизлайки видео,
#      - лайки и дизлайки к комментариям к видео
from pyspark.sql.functions import *
videos_s = (
    videos.select('video_id', 'views', 'likes', 'dislikes', 'category_id', 'tags')
          .join(comments
                .select('video_id', col('likes').alias('likes_c'),  # col('dislikes').alias('dislikes_c')  - не нашёл 🤔
                ),
                on='video_id', how='left'
            )
          .select('video_id', 'views', 'likes', 'dislikes', 'category_id', 'tags', coalesce(col('likes_c'), lit(0)).alias('likes_c'))
)
videos_s.cache().show(3, False)

+-----------+-----+-----+--------+-----------+------+-------+
|video_id   |views|likes|dislikes|category_id|tags  |likes_c|
+-----------+-----+-----+--------+-----------+------+-------+
|4yCkkOvIkUI|2306 |7    |1       |24         |[none]|5      |
|4yCkkOvIkUI|2306 |7    |1       |24         |[none]|3      |
|4yCkkOvIkUI|2306 |7    |1       |24         |[none]|6      |
+-----------+-----+-----+--------+-----------+------+-------+
only showing top 3 rows



In [119]:
# 1) score = (likes - dislikes + likes-comments) / views 
scored_videos = videos_s.withColumn('score', (col('likes') - col('dislikes') + col('likes_c')) / col('views'))
scored_videos.orderBy(desc('score')).cache().show(5, False)
scored_videos.where('score > 0').orderBy(asc('score')).cache().show(5, False)
scored_videos.cache().count()

+-----------+------+------+--------+-----------+--------------------------------+-------+------------------+
|video_id   |views |likes |dislikes|category_id|tags                            |likes_c|score             |
+-----------+------+------+--------+-----------+--------------------------------+-------+------------------+
|EUoe7cf0HYw|497846|160690|5323    |10         |Taylor Swift|Gorgeous|reputation|26     |0.3121306588784484|
|EUoe7cf0HYw|497846|160690|5323    |10         |Taylor Swift|Gorgeous|reputation|7      |0.3120924944661602|
|EUoe7cf0HYw|497846|160690|5323    |10         |Taylor Swift|Gorgeous|reputation|7      |0.3120924944661602|
|EUoe7cf0HYw|497846|160690|5323    |10         |Taylor Swift|Gorgeous|reputation|6      |0.3120904858128819|
|EUoe7cf0HYw|497846|160690|5323    |10         |Taylor Swift|Gorgeous|reputation|5      |0.3120884771596036|
+-----------+------+------+--------+-----------+--------------------------------+-------+------------------+
only showing top 5 

2945294

In [112]:
# Research file with categories:..
!head -n 22 datasets/US_category_id.json
!echo ...

{
 "kind": "youtube#videoCategoryListResponse",
 "etag": "\"m2yskBQFythfE4irbTIeOgYYfBU/S730Ilt-Fi-emsQJvJAAShlR6hM\"",
 "items": [
  {
   "kind": "youtube#videoCategory",
   "etag": "\"m2yskBQFythfE4irbTIeOgYYfBU/Xy1mB4_yLrHy_BmKmPBggty2mZQ\"",
   "id": "1",
   "snippet": {
    "channelId": "UCBR8-60-B28hp2BmDPdntcQ",
    "title": "Film & Animation",
    "assignable": true
   }
  },
  {
   "kind": "youtube#videoCategory",
   "etag": "\"m2yskBQFythfE4irbTIeOgYYfBU/UZ1oLIIz2dxIhO45ZTFR3a3NyTA\"",
   "id": "2",
   "snippet": {
    "channelId": "UCBR8-60-B28hp2BmDPdntcQ",
    "title": "Autos & Vehicles",
    "assignable": true
...


# https://chat.openai.com/c/fac3b2e7-81c4-4ef5-9cdd-120a459b0fcf
how to exctract to spark dataframe with 2 cols:
- category_id – from path: "items"."id" where "kind" = "youtube#videoCategory"
- category_name – from path: "items"."snippet"."title" for category_id

from this file:
{ "kind": "youtube#videoCategoryListResponse",
 "etag": "\"m2yskBQFythfE4irbTIeOgYYfBU/S730Ilt-Fi-emsQJvJAAShlR6hM\"",..

In [104]:
# 2) read categories from json
df = spark.read.option("multiline", "true").json("datasets/US_category_id.json")

df_exploded = df.select(explode(df.items).alias("item"))

categories_df = df_exploded.select(
    df_exploded["item.id"].alias("category_id"),
    df_exploded["item.snippet.title"].alias("category_name")
)

categories_df = categories_df.filter(df_exploded["item.kind"] == "youtube#videoCategory")

categories_df.cache().show(5, False)

+-----------+----------------+
|category_id|category_name   |
+-----------+----------------+
|1          |Film & Animation|
|2          |Autos & Vehicles|
|10         |Music           |
|15         |Pets & Animals  |
|17         |Sports          |
+-----------+----------------+
only showing top 5 rows



In [142]:
# 2) categories_score - по категориям, в котором поля: 
#  - Название категории (не id) -  в US_category_id.json
#  - Медиана показателя score из scored_videos по каждой категории

# Join scored_videos and categories_df on category_id
##videos_with_categories = scored_videos.join(categories_df, scored_videos.category_id == categories_df.category_id)
videos_with_categories = scored_videos.join(categories_df, on='category_id', how='left')

# Compute median score
median_scores = videos_with_categories.groupby("category_name").agg(
    expr('percentile_approx(score, 0.5)').alias('median_score')
)

median_scores.cache().show(11, False)

+---------------------+--------------------+
|category_name        |median_score        |
+---------------------+--------------------+
|Shows                |0.012471243491948178|
|Education            |0.03368260385297636 |
|Gaming               |0.0178513790118479  |
|Entertainment        |0.024058260052228886|
|Travel & Events      |0.02688961421740934 |
|Science & Technology |0.02789074214986546 |
|Sports               |0.010208639060805207|
|Howto & Style        |0.048267083261007926|
|Nonprofits & Activism|0.00800941219059686 |
|Film & Animation     |0.022790679555816682|
|People & Blogs       |0.03503300560376566 |
+---------------------+--------------------+
only showing top 11 rows



In [145]:
videos_df = videos_with_categories
videos_df.show(5, 44)

+-----------+-----------+-----+-----+--------+--------------------------------------------+-------+--------------------+--------------------+
|category_id|   video_id|views|likes|dislikes|                                        tags|likes_c|               score|       category_name|
+-----------+-----------+-----+-----+--------+--------------------------------------------+-------+--------------------+--------------------+
|         28|CAQ2wWVlOuc|25541|  510|      94|Tech Insider|TI|Tech|Science|Innovation|D...|    278| 0.02717199796405779|Science & Technology|
|         28|CAQ2wWVlOuc|25541|  510|      94|Tech Insider|TI|Tech|Science|Innovation|D...|    144|0.021925531498375162|Science & Technology|
|         28|CAQ2wWVlOuc|25541|  510|      94|Tech Insider|TI|Tech|Science|Innovation|D...|     83| 0.01953721467444501|Science & Technology|
|         28|CAQ2wWVlOuc|25541|  510|      94|Tech Insider|TI|Tech|Science|Innovation|D...|    133|0.021494851415371365|Science & Technology|
|     

In [139]:
# spark.sql('drop table scored_videos')
videos_df.printSchema()

root
 |-- video_id: string (nullable = true)
 |-- views: integer (nullable = true)
 |-- likes: integer (nullable = true)
 |-- dislikes: integer (nullable = true)
 |-- category_id: integer (nullable = true)
 |-- tags: string (nullable = true)
 |-- likes_c: integer (nullable = false)
 |-- score: double (nullable = true)
 |-- category_id: string (nullable = true)
 |-- category_name: string (nullable = true)



In [147]:
# videos_df.write.saveAsTable('videos', partitionBy='category_name') #, mode='overwrite')
videos_df = spark.table('videos')
videos_df.cache().count(), \
videos_df.show(5, 55)

+-----------+-----------+-----+-----+--------+------+-------+---------------------+-------------+
|category_id|   video_id|views|likes|dislikes|  tags|likes_c|                score|category_name|
+-----------+-----------+-----+-----+--------+------+-------+---------------------+-------------+
|         24|4yCkkOvIkUI| 2306|    7|       1|[none]|      5| 0.004770164787510842|Entertainment|
|         24|4yCkkOvIkUI| 2306|    7|       1|[none]|      3|0.0039028620988725065|Entertainment|
|         24|4yCkkOvIkUI| 2306|    7|       1|[none]|      6| 0.005203816131830009|Entertainment|
|         24|4yCkkOvIkUI| 2306|    7|       1|[none]|      0|0.0026019080659150044|Entertainment|
|         24|4yCkkOvIkUI| 2306|    7|       1|[none]|      0|0.0026019080659150044|Entertainment|
+-----------+-----------+-----+-----+--------+------+-------+---------------------+-------------+
only showing top 5 rows



(2945294, None)

In [105]:
# 3) popular_tags - по самым популярным тэгам
  # - название тэга + количество видео с этим тэгом
  #   ! тэги лежат строкой в поле tags
  #     - Scala-функцию для разбиения тегов: 
  #       - Но напишите свою UDF-функцию разбиения строки на тэги
  #       - и сравните время работы с её Scala-версией.
  #         - Можно замерять своими силами,
  #           - а можно воспользоваться библиотекой timeit
  #           - функции Spark из пакета pyspark.sq.functions использовать нельзя,
  #             - нужно написать свою функцию.

In [148]:
spark.sparkContext.addJar("super_udf_lib.jar")

AttributeError: 'SparkContext' object has no attribute 'addJar'

In [2]:
import pyspark
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import SparkSession
# spark.stop(); del spark
spark = SparkSession.builder.master("local") \
    .config('spark.sql.autoBroadcastJoinThreshold', 0) \
    .config('spark.sql.adaptive.enabled', 'false') \
    .config("spark.jars", "super_udf_lib.jar") \
    .getOrCreate()
display(spark)
videos = spark.table('videos')

AnalysisException: [TABLE_OR_VIEW_NOT_FOUND] The table or view `videos` cannot be found. Verify the spelling and correctness of the schema and catalog.
If you did not qualify the name with a schema, verify the current_schema() output, or qualify the name with the correct schema and catalog.
To tolerate the error on drop use DROP VIEW IF EXISTS or DROP TABLE IF EXISTS.;
'UnresolvedRelation [videos], [], false


#### Spark UI:
- http://127.0.0.1:4040/stages/

In [181]:
!man apt
# -install scala

This system has been minimized by removing packages and content that are
not required on a system that users do not log into.

To restore this content, including manpages, you can run the 'unminimize'
command. You will still need to ensure the 'man-db' package is installed.


In [184]:
import os;os.system

<function posix.system(command)>

In [177]:
!java -version
!scala --version

openjdk version "17.0.7" 2023-04-18
OpenJDK Runtime Environment (build 17.0.7+7-Ubuntu-0ubuntu122.04.2)
OpenJDK 64-Bit Server VM (build 17.0.7+7-Ubuntu-0ubuntu122.04.2, mixed mode, sharing)
/bin/bash: line 1: scala: command not found


In [169]:
!which java

/usr/bin/java


In [170]:
%%bash
echo $PATH
export PATH=/usr/bin/java:$PATH
echo $PATH

/opt/conda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/local/spark/bin
/usr/bin/java:/opt/conda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/local/spark/bin


In [171]:
!ls super*
!jar tf super_udf_lib.jar

super_udf_lib.jar
/bin/bash: line 1: jar: command not found


In [173]:
!apt install jar

E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?


In [172]:
!which jar

In [160]:
from pyspark.sql.types import ArrayType, StringType

spark.udf.registerJavaFunction(
    "splitTags", 
    "com.example.super_udf_lib.CustomUDFs.splitTagsUDF",
    # "super_udf_lib.CustomUDFs.splitTagsUDF",
    ArrayType(StringType())
)

AnalysisException: Can not load class com.example.super_udf_lib.CustomUDFs.splitTagsUDF, please make sure it is on the classpath.

In [4]:
def get_videos_df():
    videos_df = spark.read.parquet('spark-warehouse/videos')
    videos_df.cache().count(), \
    videos_df.show(5, 55)
    return videos_df

videos = get_videos_df()

+-----------+-----------+-----+-----+--------+------+-------+---------------------+-------------+
|category_id|   video_id|views|likes|dislikes|  tags|likes_c|                score|category_name|
+-----------+-----------+-----+-----+--------+------+-------+---------------------+-------------+
|         24|4yCkkOvIkUI| 2306|    7|       1|[none]|      5| 0.004770164787510842|Entertainment|
|         24|4yCkkOvIkUI| 2306|    7|       1|[none]|      3|0.0039028620988725065|Entertainment|
|         24|4yCkkOvIkUI| 2306|    7|       1|[none]|      6| 0.005203816131830009|Entertainment|
|         24|4yCkkOvIkUI| 2306|    7|       1|[none]|      0|0.0026019080659150044|Entertainment|
|         24|4yCkkOvIkUI| 2306|    7|       1|[none]|      0|0.0026019080659150044|Entertainment|
+-----------+-----------+-----+-----+--------+------+-------+---------------------+-------------+
only showing top 5 rows



In [174]:
scored_videos.registerTempTable('scored_videos')
df = spark.sql(f"""
    select splitTags(tags) as tags
      from scored_videos
""")
df.cache()
df.limit(11).toPandas()

/usr/local/spark/python/pyspark/sql/dataframe.py:330: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


AnalysisException: [UNRESOLVED_ROUTINE] Cannot resolve function `splitTags` on search path [`system`.`builtin`, `system`.`session`, `spark_catalog`.`default`].; line 2 pos 11

In [191]:
if 'videos_df' not in globals(): videos_df = spark.table('videos_df')
videos_df.show(5, 55)

Py4JJavaError: An error occurred while calling o1195.showString.
: java.lang.IllegalStateException: SparkContext has been shutdown
	at org.apache.spark.SparkContext.runJob(SparkContext.scala:2255)
	at org.apache.spark.SparkContext.runJob(SparkContext.scala:2284)
	at org.apache.spark.SparkContext.runJob(SparkContext.scala:2303)
	at org.apache.spark.sql.execution.SparkPlan.executeTake(SparkPlan.scala:530)
	at org.apache.spark.sql.execution.SparkPlan.executeTake(SparkPlan.scala:483)
	at org.apache.spark.sql.execution.CollectLimitExec.executeCollect(limit.scala:61)
	at org.apache.spark.sql.Dataset.collectFromPlan(Dataset.scala:4177)
	at org.apache.spark.sql.Dataset.$anonfun$head$1(Dataset.scala:3161)
	at org.apache.spark.sql.Dataset.$anonfun$withAction$2(Dataset.scala:4167)
	at org.apache.spark.sql.execution.QueryExecution$.withInternalError(QueryExecution.scala:526)
	at org.apache.spark.sql.Dataset.$anonfun$withAction$1(Dataset.scala:4165)
	at org.apache.spark.sql.execution.SQLExecution$.

In [5]:
from pyspark.sql.functions import udf

# Define a Python function
def split_tags(tags_str): return tags_str.split('|')

# Create a UDF from the Python function
split_tags_udf = udf(split_tags)

# Now you can use this UDF in your DataFrame transformations
videos = videos.withColumn('tags_array', split_tags_udf(videos['tags']))

In [11]:
videos.printSchema()
videos.where(f"tags is not null and tags != '[none]'").show(5, 33)

root
 |-- category_id: integer (nullable = true)
 |-- video_id: string (nullable = true)
 |-- views: integer (nullable = true)
 |-- likes: integer (nullable = true)
 |-- dislikes: integer (nullable = true)
 |-- tags: string (nullable = true)
 |-- likes_c: integer (nullable = true)
 |-- score: double (nullable = true)
 |-- category_name: string (nullable = true)
 |-- tags_array: string (nullable = true)

+-----------+-----------+-----+-----+--------+---------------------------------+-------+--------------------+-------------+---------------------------------+
|category_id|   video_id|views|likes|dislikes|                             tags|likes_c|               score|category_name|                       tags_array|
+-----------+-----------+-----+-----+--------+---------------------------------+-------+--------------------+-------------+---------------------------------+
|         24|eHq6ZA6uKOg|37400|  362|     120|American Broadcasting Company|...|     42|0.007593582887700534|Entertainm

In [15]:
from pyspark.sql.functions import pandas_udf, PandasUDFType, explode
from pyspark.sql import DataFrame
import pandas as pd

@pandas_udf('array<string>', PandasUDFType.SCALAR)
def split_tags(tags_series: pd.Series) -> pd.Series: return tags_series.str.split('|')

def add_tags_array(df: DataFrame, tags_col: str) -> DataFrame: return df.withColumn('tags_array', split_tags(df[tags_col]))

# A function to count the number of videos for each tag
def count_videos_per_tag(df: DataFrame, tags_array_col: str) -> DataFrame: return (
    df.select(
        explode(
                df[tags_array_col]
                   ).alias('tag')
     ).groupBy('tag').count()
)

# Load your DataFrame here
# videos = spark.read...

# Example data
# video_id | title | tags
# 1        | video1| tag1|tag2|tag3
# 2        | video2| tag1|tag3
# 3        | video3| tag2|tag3|tag4

videos = add_tags_array(videos, 'tags')
videos.show(3, 33)


popular_tags = count_videos_per_tag(videos, 'tags_array')

# popular_tags will be:
# tag  | count
# tag1 | 2
# tag2 | 2
# tag3 | 3
# tag4 | 1
popular_tags.show(3, 33)

+-----------+-----------+-----+-----+--------+------+-------+---------------------+-------------+----------+
|category_id|   video_id|views|likes|dislikes|  tags|likes_c|                score|category_name|tags_array|
+-----------+-----------+-----+-----+--------+------+-------+---------------------+-------------+----------+
|         24|4yCkkOvIkUI| 2306|    7|       1|[none]|      5| 0.004770164787510842|Entertainment|  [[none]]|
|         24|4yCkkOvIkUI| 2306|    7|       1|[none]|      3|0.0039028620988725065|Entertainment|  [[none]]|
|         24|4yCkkOvIkUI| 2306|    7|       1|[none]|      6| 0.005203816131830009|Entertainment|  [[none]]|
+-----------+-----------+-----+-----+--------+------+-------+---------------------+-------------+----------+
only showing top 3 rows

+-----------+-----+
|        tag|count|
+-----------+-----+
|ABC Network|  710|
|film review| 5100|
|     Heaven| 6700|
+-----------+-----+
only showing top 3 rows



In [27]:
df = videos
df = df.withColumn('tags_array', split_tags(df['tags']))
df.cache().show(3, 33)
df = df.select('*',
            explode(
                df['tags_array']
                   ).alias('tag')
      )
df.show(33, 33)
# df = df.groupBy('tag').count()
# df = df.orderBy(desc('count'))
# df.cache().show(33, 33)

+-----------+-----------+-----+-----+--------+------+-------+---------------------+-------------+----------+
|category_id|   video_id|views|likes|dislikes|  tags|likes_c|                score|category_name|tags_array|
+-----------+-----------+-----+-----+--------+------+-------+---------------------+-------------+----------+
|         24|4yCkkOvIkUI| 2306|    7|       1|[none]|      5| 0.004770164787510842|Entertainment|  [[none]]|
|         24|4yCkkOvIkUI| 2306|    7|       1|[none]|      3|0.0039028620988725065|Entertainment|  [[none]]|
|         24|4yCkkOvIkUI| 2306|    7|       1|[none]|      6| 0.005203816131830009|Entertainment|  [[none]]|
+-----------+-----------+-----+-----+--------+------+-------+---------------------+-------------+----------+
only showing top 3 rows

+-----------+-----------+-----+-----+--------+------+-------+----------------------+-------------+----------+------+
|category_id|   video_id|views|likes|dislikes|  tags|likes_c|                 score|category_na

#### 4) И личная просьба от Марка: он любит котов (а кто не их не любит!) и хочет найти самые интересные комментарии (топ-5) к видео про котов. “Видео про котов” - видео, у которого есть тэг “cat”.


In [24]:
df.where(f"tag like '%cat%'").show(33, 33)

+-------------------+-----+
|                tag|count|
+-------------------+-----+
|          education|40916|
|                cat|19505|
|          cute cats|16100|
|               cats|14301|
|          funny cat|12500|
|              catch|12319|
|        educational|11620|
|          cat fails| 8700|
|        simons cats| 8700|
|      simon the cat| 8700|
|         simons cat| 8700|
|          simonscat| 8700|
|      animated cats| 8700|
|black and white cat| 8700|
|        simon's cat| 8700|
|         cat lovers| 8700|
|         funny cats| 8700|
|      TED Education| 7885|
|        application| 7300|
|         cat makeup| 7000|
|     cat eye makeup| 7000|
|      cat-headlines| 5116|
|   every cat at 3am| 4900|
|       cat freakout| 4900|
|    gus johnson cat| 4900|
|   cats compilation| 4900|
|          cat prank| 4900|
|         crazy cats| 4900|
|         cat sketch| 4900|
|            gus cat| 4900|
|           cat meme| 4900|
|        complicated| 4756|
|          spidercat

In [25]:
df = videos
df = df.cache().where(f"tag like '%cat%'")
df.show(33, 33)

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `tag` cannot be resolved. Did you mean one of the following? [`tags`, `likes`, `score`, `views`, `likes_c`].; line 1 pos 0;
'Filter 'tag LIKE %cat%
+- Project [category_id#0, video_id#1, views#2, likes#3, dislikes#4, tags#5, likes_c#6, score#7, category_name#8, split_tags(tags#5)#1901 AS tags_array#1902]
   +- Project [category_id#0, video_id#1, views#2, likes#3, dislikes#4, tags#5, likes_c#6, score#7, category_name#8, split_tags(tags#5)#1643 AS tags_array#1644]
      +- Project [category_id#0, video_id#1, views#2, likes#3, dislikes#4, tags#5, likes_c#6, score#7, category_name#8, split_tags(tags#5)#520 AS tags_array#521]
         +- Relation [category_id#0,video_id#1,views#2,likes#3,dislikes#4,tags#5,likes_c#6,score#7,category_name#8] parquet
